In [1439]:
from module import *

In [1440]:
class MatrixFactorizationCF(object):
    def __init__(self, data, K, lam = 0.1, Xinit = None, Winit = None, learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        
        # data: utility matrix
        self.data = data

        # K: number of latent features
        self.K = K

        # lam: regularization parameter
        self.lam = lam

        # learning_rate: alpha
        self.learning_rate = learning_rate

        # max_iter: number of iterations
        self.max_iter = max_iter

        # print_every: print loss every print_every iterations
        self.print_every = print_every

        # user_based: 1 if we are working with user-based CF, 0 if item-based
        self.user_based = user_based

        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(data[:, 0])) + 1
        self.n_items = int(np.max(data[:, 1])) + 1

        # number of ratings
        self.n_ratings = data.shape[0]

        # Xinit: item matrix initialization with row = number of items, col = latent features
        if Xinit is None:
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit

        # Winit: user matrix initialization with col = number of users, row = latent features
        if Winit is None:
            self.W = np.random.randn(K, self.n_users)
        else:
            self.W = Winit

        # copied data
        self.data_n = data.copy()

    def normalize(self):
        # if we are working with user-based CF
        if self.user_based:
            user_col = 0
            item_col = 1
            n_object = self.n_users
        
        # if we are working with item-based CF
        else:
            user_col = 1
            item_col = 0
            n_object = self.n_items

        users = self.data[:, user_col]
        # init mean matrix
        self.mu = np.zeros((n_object,))
        
        for n in range(n_object):
            # get all ratings of user n
            ids = np.where(users == n)[0].astype(int)
            item_ids = self.data[ids, item_col]
            ratings = self.data[ids, 2]

            # calculate mean
            m = np.mean(ratings)
            if np.isnan(m):
                m = 0
            self.mu[n] = m

            # normalize
            self.data_n[ids, 2] = ratings - self.mu[n]
        return self.data_n

    def loss(self):
        L = 0
        # for each rating in data
        for i in range(self.n_ratings):
            # get user, item, rating
            n, m, _rating = int(self.data_n[i, 0]), int(self.data_n[i, 1]), self.data_n[i, 2]
            # UPDATE LOSS
            xm = self.X[m, :]
            wn = self.W[:, n]
            L += 0.5 * (_rating - xm.dot(wn))**2
        
        L /= self.n_ratings
        # regularization
        L += .5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L

    def get_items_rated_by_user(self, user_id):
        # get all items which are rated by user n
        ids = np.where(self.data[:, 0] == user_id)[0]
        item_ids = self.data[ids, 1].astype(int)
        ratings = self.data[ids, 2]
        return (item_ids, ratings)
    
    def get_users_who_rate_item(self, item_id):
        # get all users who rated item m
        ids = np.where(self.data[:, 1] == item_id)[0]
        user_ids = self.data[ids, 0].astype(int)
        ratings = self.data[ids, 2]
        return (user_ids, ratings)
    
    '''
        Update X
            The method optimize item matrix X, given user matrix W
    '''
    def updateX(self):
        # Walk through each item to update
        for m in range(self.n_items):
            # Get all users who rated item m and rating of them
            user_ids, ratings = self.get_users_who_rate_item(m)
            
            # Get Wm matrix related to user_ids
            Wm = self.W[:, user_ids]

            xm = self.X[m, :]
            # Calculate gradient
            grad_xm = -(ratings - xm.dot(Wm)).dot(Wm.T)/self.n_ratings + self.lam*xm
            
            # Update X
            xm -= self.learning_rate*grad_xm.reshape((self.K,))
            self.X[m, :] = xm
    
    def updateW(self):
        # Walk through each user to update
        for n in range(self.n_users):
            # Get all items rated by user n and rating of them
            item_ids, ratings = self.get_items_rated_by_user(n)
            
            # Get Xn matrix related to item_ids
            Xn = self.X[item_ids, :]

            wn = self.W[:, n]
            # Calculate gradient
            grad_wn = -Xn.T.dot(ratings - Xn.dot(wn))/self.n_ratings + self.lam*wn
            
            # Update W
            wn -= self.learning_rate*grad_wn.reshape((self.K,))
            self.W[:, n] = wn

    def predict(self, u, i):
        u = int(u)
        i = int(i)

        # create bias to make rate in range [0, 5]
        if self.user_based:
            bias = self.mu[u]
        else:
            bias = self.mu[i]
        
        # compute the pred rate y_hat = x*w
        pred = self.X[i, :].dot(self.W[:, u]) + bias

        if pred < 0 :
            return 0
        if pred > 5: 
            return 5
        return pred
    
    def predict_for_user(self, user_id):
        # get all items which are rated by user
        ids = np.where(self.data[:, 0] == user_id)[0]
        items_rated_by_u = self.data[ids, 1].tolist()
        
        # predict ratings for items that are not rated by user
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings = []

        # return list of (item, rating) of items that are not rated by user
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        return predicted_ratings

    def RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.predict(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2
        RMSE = np.sqrt(SE/n_tests)
        return RMSE

    def fit(self):
        self.normalize()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                print('iter =', it + 1, ', loss =', self.loss())

    def score(self):
        pass

## __Hàm chức năng sử dụng cho quá trình__:

### __Hàm chức năng cho ma trận__

In [1441]:
def tranpose_matrix(matrix):
    if len(matrix.shape) < 2:
        row, col = 1, matrix.shape[0]
        matrix = np.array([matrix])
    else: 
        row, col = matrix.shape
    new_matrix = np.zeros((col, row))
    for i in range(row):
        for j in range(col):
            new_matrix[j][i] = matrix[i][j]
    return new_matrix

In [1442]:
def multiple_matrix(A, B):
    dim_a = A.shape
    dim_b = B.shape

    if len(dim_a) < 2:
        dim_a = (1, dim_a[0])
    if len(dim_b) < 2:
        dim_b = (1, dim_b[0])

    if dim_a[1] != dim_b[0]:
        return KeyError("Matrix A and B can't be multiplied")
    
    result = np.zeros((dim_a[0], dim_b[1]))
    for i in range(dim_a[0]):
        for j in range(dim_b[1]):
            for k in range(dim_a[1]):
                result[i][j] += A[i][k]*B[k][j]

    return result

### __Hàm tìm các items được rate bởi các user__

__Mục đích__: Tìm tất cả cặp (items, rates) mà các items này được đánh giá bởi user đầu vào.<br>
__Đầu vào__: user_id mà cần để truy xuất các giá trị.

+ Hàm `where_user_in_data` để tìm __index__ của các user có trong data ban đầu.
+ Hàm `get_items_rated_by_user` truy xuất các items được rate và giá trị rate.

In [1443]:
def where_user_in_data(data, user_id):
    valid_index = []

    for i in range(data.shape[0]):
        if data[i][0] == user_id:
            valid_index.append(i)
    return np.array(valid_index)

def get_items_rated_by_user(data, user_id):
    idx = where_user_in_data(data, user_id)
    items_ids = []
    ratings = []
    for i in idx:
        items_ids.append(data[i][1])
        ratings.append(data[i][2])
    return (np.array(idx), np.array(items_ids), np.array(ratings))

### __Hàm tìm các users đã rated cho item__:

__Mục đích__: Tìm tất cả cặp (users, rates) mà các users này đánh giá cho item đầu vào.<br>
__Đầu vào__: item_id mà cần để truy xuất các giá trị.

In [1444]:
def where_item_in_data(data, item_id):
    valid_index = []

    for i in range(data.shape[0]):
        if data[i][1] == item_id:
            valid_index.append(i)
    return np.array(valid_index)

def get_users_who_rate_item(data, item_id):
    idx = where_item_in_data(data, item_id)
    user_ids = []
    ratings = []
    for i in idx:
        user_ids.append(data[i][0])
        ratings.append(data[i][2])
    return (np.array(idx), np.array(user_ids), np.array(ratings))

## __Normalize__

In [1445]:
def swap_cols(data, frm, to):
    data[:,[frm, to]] = data[:,[to, frm]]

def normalize(data, user_based = 1):
    # create copy_data
    cdata = data.copy()
    if user_based == 0:
        swap_cols(cdata, 0, 1)
    user_col = 0
    item_col = 1
    n_object = int(np.max(cdata[:, user_col])) + 1
    
    users = cdata[:, user_col]
    mu = np.zeros((n_object,))

    for n in range(n_object):
        idx, items_ids, ratings = get_items_rated_by_user(cdata, n)

        m = np.mean(ratings)
        if np.isnan(m):
            m = 0
        mu[n] = m

        cdata[list(idx), 2] = ratings - mu[n]
    return cdata, mu


## __Update hàm Loss__

In [1446]:
def frobenius_norm_without_numpy(X):
    # Initialize sum of squares to zero
    sum_of_squares = 0
    
    # Iterate through each row of the matrix
    for row in X:
        # Iterate through each element in the row
        for element in row:
            # Add the square of the element to the sum of squares
            sum_of_squares += element ** 2
    
    # Take the square root of the sum of squares to get the Frobenius norm
    frobenius_norm = sum_of_squares ** 0.5
    
    return frobenius_norm

In [1447]:
def loss(data, X, W, lam):
    L = 0
    for i in range(data.shape[0]):
        # get user, item, rating
        n, m, _rating = int(data[i, 0]), int(data[i, 1]), data[i, 2]
        # Update loss
        xm = np.array([X[m, :]])
        wn = W[:, n].reshape((W.shape[0], 1))
        
        multi_result = multiple_matrix(xm, wn)[0][0]
        
        L += 0.5 * (_rating - multi_result)**2
    L /= data.shape[0]
    # # regularization
    L += .5*lam*(frobenius_norm_without_numpy(X) + frobenius_norm_without_numpy(W))
    return L

In [1448]:
def updateX(data, X, W, lam, learning_rate):
    for m in range(X.shape[0]):
        # Get all users who rated item m and rating of them
        idx, user_ids, ratings = get_users_who_rate_item(data, m)
        Wm = W[:, list(user_ids)].reshape((W.shape[0], len(user_ids)))
        xm = X[m, :].reshape((1, X.shape[1]))

        # Calculate gradient
        xm_dot_Wm = multiple_matrix(xm, Wm)
        reduce_ratings = (ratings - xm_dot_Wm)
        tWm = tranpose_matrix(Wm)
        grad_xm = -multiple_matrix(reduce_ratings, tWm)/data.shape[0] + lam*xm
        # Update X
        xm -= learning_rate*grad_xm.reshape((X.shape[1],))
        X[m, :] = xm
    return X

def updateW(data, X, W, lam, learning_rate):
    for n in range(W.shape[1]):
        # Get all items rated by user n and rating of them
        idx, item_ids, ratings = get_items_rated_by_user(data, n)
        Xn = X[list(item_ids), :].reshape((len(item_ids), X.shape[1]))
        wn = W[:, n].reshape((W.shape[0], 1))
        # Calculate gradient
        Xn_dot_wn = multiple_matrix(Xn, wn)
        reduce_ratings = (tranpose_matrix(ratings) - Xn_dot_wn)
        tXn = tranpose_matrix(Xn)
        grad_wn = -multiple_matrix(tXn, reduce_ratings)/data.shape[0] + lam*wn

        wn -= learning_rate*grad_wn
        W[:, n] = wn.reshape((W.shape[0],))
    return W

In [1449]:
a = np.array([[1],[2]])
5-a

array([[4],
       [3]])

In [1450]:
def predict(data, X, W, mu, user_id, item_id):
    u = int(user_id)
    i = int(item_id)
    # create bias to make rate in range [0, 5]
    bias = mu[u]
    # compute the pred rate y_hat = x*w
    pred = X[i, :].dot(W[:, u]) + bias
    if pred < 0 :
        return 0
    if pred > 5: 
        return 5
    return pred

In [1451]:
def predict_for_user(data, X, W, mu, user_id):
    # get all items which are rated by user
    idx, items_rated_by_u, _ = get_items_rated_by_user(data, user_id)
    # predict ratings for items that are not rated by user
    y_pred = X.dot(W[:, user_id]) + mu[user_id]
    predicted_ratings = []
    # return list of (item, rating) of items that are not rated by user
    for i in range(X.shape[0]):
        if i not in items_rated_by_u:
            predicted_ratings.append((i, y_pred[i]))
    return predicted_ratings

In [1452]:
def RMSE(data, X, W, mu, rate_test):
    n_tests = rate_test.shape[0]
    SE = 0 # squared error
    for n in range(n_tests):
        pred = predict(data, X, W, mu, rate_test[n, 0], rate_test[n, 1])
        SE += (pred - rate_test[n, 2])**2
    RMSE = np.sqrt(SE/n_tests)
    return RMSE

In [1453]:
def fit(data, X, W, lam = 0.1, learning_rate = 0.5, max_iter = 10, print_every = 1):
    data, mu = normalize(data)
    for it in range(max_iter):
        X = updateX(data, X, W, lam, learning_rate)
        W = updateW(data, X, W, lam, learning_rate)
        if (it + 1) % print_every == 0:
            print('iter =', it + 1, ', loss =', loss(data, X, W, lam))
    return X, W, mu

In [1454]:
def score(data, X, W, mu, rate_test):
    return RMSE(data, X, W, mu, rate_test)

In [1455]:
def run_mfcf(data, X, W, lam = 0.1, learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1, rate_test = None):
    X, W = fit(data, X, W, lam, learning_rate, max_iter, print_every)
    data, mu = normalize(data)
    return score(data, X, W, mu, rate_test)

In [1456]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('../../data/ml-100k/ub.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('../../data/ml-100k/ub.test', sep='\t', names=r_cols)

rate_train = np.array(ratings_base, dtype= object)
rate_test = np.array(ratings_test, dtype= object)

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [1457]:
# get number of users id < 100
n_users = 100
rate_train = rate_train[rate_train[:, 0] < n_users]

In [1458]:
rate_train

array([[0, 0, 5, 874965758],
       [0, 1, 3, 876893171],
       [0, 2, 4, 878542960],
       ...,
       [99, 1234, 4, 891375454],
       [99, 1236, 3, 891375630],
       [99, 1237, 2, 891375068]], dtype=object)

In [1459]:
rs = MatrixFactorizationCF(rate_train, K = 10, lam = .1, print_every = 2, learning_rate = 0.5, max_iter = 100, user_based = 1)

In [1460]:
normalize_matrix, mu = normalize(rate_train, user_based=1)

In [1461]:
normalize_matrix

array([[0, 0, 1.4122137404580153, 874965758],
       [0, 1, -0.5877862595419847, 876893171],
       [0, 2, 0.4122137404580153, 878542960],
       ...,
       [99, 1234, 1.0, 891375454],
       [99, 1236, 0.0, 891375630],
       [99, 1237, -1.0, 891375068]], dtype=object)

In [1464]:
X, W, mu = fit(rate_train, rs.X, rs.W, max_iter=50, print_every=10)

iter = 10 , loss = 5.373752160427767
iter = 20 , loss = 3.183015841135576
iter = 30 , loss = 2.105349731422197
iter = 40 , loss = 1.4881120518174111
iter = 50 , loss = 1.121825882477677


In [1465]:
rate_test[3]

array([0, 89, 4, 878542300], dtype=object)

In [1466]:
predict_for_user(rate_train, X, W, mu, 3)

[(0, 4.285671210640799),
 (1, 4.253409433085062),
 (2, 4.30679321777691),
 (3, 4.274720863802698),
 (4, 4.283622778513977),
 (5, 4.301277881222039),
 (6, 4.2688785574525845),
 (7, 4.296268935231834),
 (8, 4.250213108331623),
 (9, 4.288943321780295),
 (10, 4.30343981222352),
 (11, 4.286992971382361),
 (12, 4.302697601352284),
 (13, 4.301441853958295),
 (14, 4.274958239240785),
 (15, 4.284696586338504),
 (16, 4.275089058484753),
 (17, 4.315326474476799),
 (18, 4.2935824080814),
 (19, 4.285241283116367),
 (20, 4.306676873085573),
 (21, 4.287237544813799),
 (22, 4.266933723276422),
 (23, 4.285110197839473),
 (24, 4.314042534980857),
 (25, 4.287952816106307),
 (26, 4.264910163268788),
 (27, 4.254344450511343),
 (28, 4.274678844083234),
 (29, 4.241972281642043),
 (30, 4.317323380372415),
 (31, 4.301907494988643),
 (32, 4.2813475910177194),
 (33, 4.310326714840678),
 (34, 4.3013289913693535),
 (35, 4.296147415186583),
 (36, 4.26919533776478),
 (37, 4.292772243102734),
 (38, 4.267149597005614)